In [75]:
import dask.dataframe as dd

In [76]:
ddf = dd.read_csv(r'D:\DaskData\WeatherEvents.csv',dtype={"ZipCode": "Int64"})

In [77]:
#Getting a sneak peak in the data
ddf.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149


In [78]:
#To get the datatypes of columns
ddf._meta.dtypes

EventId              string[pyarrow]
Type                 string[pyarrow]
Severity             string[pyarrow]
StartTime(UTC)       string[pyarrow]
EndTime(UTC)         string[pyarrow]
Precipitation(in)            float64
TimeZone             string[pyarrow]
AirportCode          string[pyarrow]
LocationLat                  float64
LocationLng                  float64
City                 string[pyarrow]
County               string[pyarrow]
State                string[pyarrow]
ZipCode                        Int64
dtype: object

In [79]:
ddf

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
npartitions=16,,,,,,,,,,,,,,
,string,string,string,string,string,float64,string,string,float64,float64,string,string,string,Int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [80]:
ddf["County"].value_counts().compute()

County
Alachua       4984
Andrews       2125
Arlington     4413
Barrow        3833
Barton        2758
              ... 
Tehama        2366
Tishomingo    5427
Vigo          4982
Whatcom       5067
Yuba          8206
Name: count, Length: 1100, dtype: int64

In [81]:
ddf["Type"].value_counts().compute()

Type
Fog              2014190
Cold              232192
Storm              61317
Precipitation     157186
Hail                2927
Rain             5000794
Snow             1158575
Name: count, dtype: int64

In [82]:
#Using Compute
ddf_andrews = ddf[ddf["County"] == "Andrews"]
res_snow = ddf_andrews.groupby("Type")["EventId"].count().compute()
print(res_snow)
print(type(res_snow))

Type
Fog               384
Precipitation      33
Rain             1571
Cold               59
Snow               57
Storm              21
Name: EventId, dtype: int64
<class 'pandas.core.series.Series'>


In [83]:
#Using Persist
ddf_andrews = ddf[ddf["County"] == "Andrews"].persist()
res_snow = ddf_andrews.groupby("Type")["EventId"].count().compute()
print(res_snow)
print(type(ddf_andrews))

Type
Fog               384
Precipitation      33
Rain             1571
Cold               59
Snow               57
Storm              21
Name: EventId, dtype: int64
<class 'dask_expr._collection.DataFrame'>


In [84]:
#Taking only two partitions of the data
sliced_data = ddf.partitions[3:5]
print("Sliced data no of partitions: ",sliced_data.npartitions)
print("Original data no of partitions: ",ddf.npartitions)
print("andrews data no of partitions: ",ddf.npartitions)

Sliced data no of partitions:  2
Original data no of partitions:  16
andrews data no of partitions:  16


In [85]:
#Repartition andrews data
ddf_andrews = ddf_andrews.repartition(npartitions=10)
print(ddf_andrews.npartitions)

10


In [100]:
meta = {
    'EventId': 'object',                 
    'Type': 'object',                     
    'Severity': 'object',                 
    'StartTime(UTC)': 'object',          
    'EndTime(UTC)': 'object',             
    'Precipitation(in)': 'float64',       
    'TimeZone': 'object',                
    'AirportCode': 'object',             
    'LocationLat': 'float64',            
    'LocationLng': 'float64',            
    'City': 'object',                    
    'County': 'object',                  
    'State': 'object',                   
    'ZipCode': 'Int64',
    'StartDate': "object",
    'StartTime': "object",
    'EndDate': "object",
    'EndTime': "object"
}

#Applying a function to all data partitions
def split_date(ddf):
    ddf[["StartDate","StartTime"]] = ddf["StartTime(UTC)"].str.split(' ',expand=True)
    ddf[["EndDate","EndTime"]] = ddf["EndTime(UTC)"].str.split(' ',expand=True)
    return ddf
ddf2 = ddf.map_partitions(split_date, meta=meta).compute()
ddf2.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),Precipitation(in),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,StartDate,StartTime,EndDate,EndTime
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-06,23:14:00,2016-01-07,00:34:00
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-07,04:14:00,2016-01-07,04:54:00
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,0.03,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-07,05:54:00,2016-01-07,15:34:00
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-08,05:34:00,2016-01-08,05:54:00
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,0.00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149,2016-01-08,13:54:00,2016-01-08,15:54:00
